In [1]:
#https://www.tensorflow.org/get_started/mnist/beginners

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
plt.imshow(np.reshape(mnist.train.images[0],(28,28)))
plt.show()

In [ ]:
x = tf.placeholder(tf.float32, [None, 784])

In [ ]:
with tf.device('/gpu:0'):
    W = tf.Variable(tf.zeros([784, 10]))
    b = tf.Variable(tf.zeros([10]))

In [ ]:
#y = tf.nn.softmax(tf.matmul(x, W) + b)
with tf.device('/gpu:0'):
    y = tf.nn.softmax(tf.tanh(tf.matmul(x, W) + b))

In [ ]:
#True classes
with tf.device('/gpu:0'):
    y_ = tf.placeholder(tf.float32, [None, 10])

In [ ]:
#cross_entropy
with tf.device('/gpu:0'):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [ ]:
with tf.device('/gpu:0'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

In [ ]:
#Session executing model
#workarounds for GPU memory in jupyter notebook
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
config = tf.ConfigProto(log_device_placement=True, gpu_options=gpu_options)
config.gpu_options.allow_growth=True
sess = tf.InteractiveSession(config=config)
tf.global_variables_initializer().run()

In [ ]:
accs = []
for _ in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(30000)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    #cp = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    #acc = tf.reduce_mean(tf.cast(cp, tf.float32))
    #accs.append(acc)

In [ ]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

In [ ]:
plot(accs)

In [4]:
#Session executing model
#workarounds for GPU memory in jupyter notebook
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
config = tf.ConfigProto(log_device_placement=True, gpu_options=gpu_options)
config.gpu_options.allow_growth=True
sess = tf.InteractiveSession(config=config)
tf.global_variables_initializer().run()

In [21]:
hidden_size = 784*3
with tf.device('/gpu:0'):
    x = tf.placeholder(tf.float32, [None, 784])
    y_ = tf.placeholder(tf.float32, [None, 10])
    
    W1 = tf.Variable(tf.zeros([784, hidden_size]))
    W2 = tf.Variable(tf.zeros([hidden_size, 10]))
    b1 = tf.Variable(tf.zeros([hidden_size]))
    b2 = tf.Variable(tf.zeros([10]))
    
    hid = tf.nn.softmax(tf.tanh(tf.matmul(x, W1) + b1))
    y = tf.nn.softmax(tf.tanh(tf.matmul(hid, W2) + b2))
    
    #W1 = tf.Variable(tf.zeros([784, 10]))
    #y = tf.nn.softmax(tf.tanh(tf.matmul(x, W1) + b2))
    
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)
tf.global_variables_initializer().run()

In [22]:
accs = []
for _ in range(10):
    batch_xs, batch_ys = mnist.train.next_batch(30000)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [23]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.1135
